<h1>Run Enriched Polygons Through Trained OSM Completeness Model</h1>
<p>After enriching a collection of 250-m by 250-m grid cells, the output can be run through the trained OSM completeness model to produce predictions of OSM building footprint area.</p>

In [ ]:
import json
import joblib
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, box, mapping, Point, Polygon
from matplotlib.patches import Polygon as mpoly
import matplotlib
import cartopy.crs as ccrs
import cartopy
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import numpy as np
import glob

<h2>These are all variables that need to be set before running the notebook</h2>

In [ ]:
# file containing trained model .sav file from training notebook
trainedModel = 'path_to_trained_model.sav'

# load model
model = joblib.load(trainedModel)

#predicted area value below which a cell will be considered "no built up area"
noRoadThresh=1000

In [ ]:
def returnFeatureVals(featureString, variable):
    return [x['properties'][featureString] for x in variable['features']]

In [ ]:
#file containing enriched cells on which to run the trained model
filename = [f for f in glob.glob("path_to_enriched*.json")]

# what output json file should be called (cells containing predicted area and completeness values)
outputFilename = []
for i in enumerate(filename):
    outputFilename.append("outputPath"+str(i[0])+".json")
    
#directory for images to be output
outputDirectory = 'path_to_outputDirectory'

# lists for calculations
mapped_list = []
mapped_need_list = []

In [ ]:
for inputFile,file, outputFile in zip(enumerate(filename), filename, enumerate(outputFilename)):
    
    # 1. load data
    with open(file,'r') as data:
        x = json.load(data)
        
    # 2. create polygons for geodataframe
    polygons=[]
    for feature in x['features']:
        g = shape(feature['geometry']).buffer(0)
        polygons.append(g)
    print(len(polygons))
    
    # function
    def returnFeatureVals(featureString, variable):
        return [x['properties'][featureString] for x in variable['features']]

    # 3. set feature dictionary
    applyFeatureDict = {
        'ndbi':returnFeatureVals('ndbi', x),
        'ndvi':returnFeatureVals('ndvi', x),
        'savi':returnFeatureVals('savi', x),
        'ui':returnFeatureVals('ui', x),
        'viirs':returnFeatureVals('viirs', x),
        'slope':returnFeatureVals('slope', x),
        'texture':returnFeatureVals('texture', x),
        'forest':returnFeatureVals('forest', x),
        'ndbi_diff2018':returnFeatureVals('ndbi_diff2018', x),
        'ndvi_diff2018':returnFeatureVals('ndvi_diff2018', x),
        'savi_diff2018':returnFeatureVals('savi_diff2018', x),
        'ui_diff2018':returnFeatureVals('ui_diff2018', x),
        'ndbi_diff2017':returnFeatureVals('ndbi_diff2017', x),
        'ndvi_diff2017':returnFeatureVals('ndvi_diff2017', x),
        'savi_diff2017':returnFeatureVals('savi_diff2017', x),
        'ui_diff2017':returnFeatureVals('ui_diff2017', x),
        'ndbi_diff2016':returnFeatureVals('ndbi_diff2016', x),
        'ndvi_diff2016':returnFeatureVals('ndvi_diff2016', x),
        'savi_diff2016':returnFeatureVals('savi_diff2016', x),
        'ui_diff2016':returnFeatureVals('ui_diff2016', x),
        'ndbi_diff2015':returnFeatureVals('ndbi_diff2015', x),
        'ndvi_diff2015':returnFeatureVals('ndvi_diff2015', x),
        'savi_diff2015':returnFeatureVals('savi_diff2015', x),
        'ui_diff2015':returnFeatureVals('ui_diff2015', x),
        'MERIT':returnFeatureVals('MERIT', x),
        'ndbiUSGS':returnFeatureVals('ndbiUSGS', x),
        'ndviUSGS':returnFeatureVals('ndviUSGS', x),
        'saviUSGS':returnFeatureVals('saviUSGS', x),
        'PopDen':returnFeatureVals('PopDen', x),
        'PopDenUN':returnFeatureVals('PopDenUN', x), 
        'length':returnFeatureVals('length',x)
    }
    
    # 4. Apply dict
    applyDF = pd.DataFrame.from_dict(applyFeatureDict)
    applyGeoDF = gpd.GeoDataFrame(applyDF,crs = 4326, geometry=polygons)
    applyGeoDF = applyGeoDF.fillna(0)
    
    # 5. apply features
    applyFeatureDF = applyGeoDF[['ndbi',
                                   'ndvi',
                                   'savi',
                                   'ui',
                                   'ndbi_diff2018', 
                                    'ndvi_diff2018', 
                                    'savi_diff2018', 
                                    'ui_diff2018', 
                                    'ndbi_diff2017',
                                    'ndvi_diff2017',
                                    'savi_diff2017',
                                    'ui_diff2017',
                                    'ndbi_diff2016',
                                    'ndvi_diff2016',
                                    'savi_diff2016',
                                    'ui_diff2016',
                                    'ndbi_diff2015',
                                    'ndvi_diff2015',
                                    'savi_diff2015',
                                    'ui_diff2015',
                                   'viirs',
                                   'slope',
                                   'texture',
                                   'forest',
                                   'MERIT', 
                                   'ndbiUSGS', 
                                   'ndviUSGS', 
                                   'saviUSGS', 
                                   'PopDen', 
                                   'PopDenUN'
                                   ]]
    applyTargetDF = applyGeoDF['length']
    
    # 5. apply model
    y_apply = model.predict(applyFeatureDF)

    
    # 6. generate data
    for i,feature in enumerate(x['features']):
        length = x['features'][i]['properties']['length']
        #erase other properties for smaller output file
        feature['properties'] = {}
        #predicted OSM road footprint area
        feature['properties']['plength'] = y_apply[i]
        #actual mapped road footprint area
        feature['properties']['builtLength'] = length
        
    # 7. save output
    with open(outputFile[1],'w') as f:
        json.dump(x,f)
        
    # 8. visualization
    #import as dictionary
    with open(outputFile[1],"r") as data:
        x = json.load(data)
    #load as dataframe
    plotdf = gpd.read_file(outputFile[1])

    bounds = plotdf.total_bounds

    bounds = [bounds[0], bounds[2], bounds[1], bounds[3]]

    clat = (bounds[2]+bounds[3])/2
    clon = (bounds[0]+bounds[1])/2

    features = x['features']
    polys=[]
    #predicted area
    vals = []
    #completeness
    vals2 = []
    lws=[]
    for i,feature in enumerate(features):
        if (i%1000)==0:
            print(f'{i} of {len(features)}')

        coords = feature['geometry']['coordinates'][0]
        x=[]
        y=[]

        vals.append(feature['properties']['plength'])

        try:
            vals2.append(feature['properties']['builtLength']/feature['properties']['plength'])
        except:
            vals2.append(0)

        for point in coords:
            x.append(point[0])
            y.append(point[1])
        transformed = ccrs.LambertConformal(central_latitude=clat,central_longitude=clon).transform_points(ccrs.PlateCarree(),np.asarray(x),np.asarray(y))
        polys.append(mpoly(transformed[:,0:2]))
        lws.append(0.05)

    cmap = matplotlib.cm.get_cmap('viridis')
    cmap2 = matplotlib.cm.get_cmap('RdYlGn')
    
    #norm = matplotlib.colors.Normalize(vmin=0,vmax=np.nanmax(vals))
    norm = matplotlib.colors.Normalize(vmin=0,vmax=4000)
    norm2 = matplotlib.colors.Normalize(vmin=0,vmax=1)
    fcs = cmap(norm(vals))
    vals2 = np.asarray(vals2)
    vals2[vals2>1] = 1
    fcs2 = cmap2(norm2(vals2))

    fcs2[np.asarray(vals)<noRoadThresh] = (.9,.9,.9,1)

    #pc=PatchCollection(polys, facecolors=fcs,edgecolors=fcs,linewidths=lws)   
    #pc2=PatchCollection(polys, facecolors=fcs2,edgecolors=fcs2,linewidths=lws)   

    pc=collections.PatchCollection(polys, facecolors=fcs,edgecolors=fcs,linewidths=lws)   
    pc2=collections.PatchCollection(polys, facecolors=fcs2,edgecolors=fcs2,linewidths=lws)   

    # image 1
    fig = plt.figure(constrained_layout = True)
    gs = fig.add_gridspec(20,10)
    ax1 = fig.add_subplot(gs[0:19,:],projection=ccrs.LambertConformal(central_latitude=clat,central_longitude=clon))
    ax2= fig.add_subplot(gs[19::,:])
    ax1.set_extent(bounds)
    ax1.set_title('Predicted Length',fontsize=10)
    ax1.add_collection(pc)
    cb=fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap),
                 cax=ax2, orientation='horizontal')
    cb.ax.tick_params(labelsize=8)
    cb.set_label(label="Length (meters)",fontsize=8)
    plt.savefig(outputDirectory+'predicted'+str(inputFile[0])+'.png',bbox_inches='tight',dpi=100) # dpi=1000
    plt.close(fig) 


    # image 2
    fig = plt.figure(constrained_layout = True)
    gs = fig.add_gridspec(20,10)
    ax1 = fig.add_subplot(gs[0:19,:],projection=ccrs.LambertConformal(central_latitude=clat,central_longitude=clon))
    ax2= fig.add_subplot(gs[19::,:])
    ax1.set_extent(bounds)
    ax1.set_title('Ratio Predicted to Actual Length',fontsize=10)
    ax1.add_collection(pc2)
    cb=fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm2, cmap=cmap2),
                 cax=ax2, orientation='horizontal')
    cb.ax.tick_params(labelsize=8)
    cb.set_label(label="Ratio",fontsize=8)
    plt.savefig(outputDirectory+'ratio'+str(inputFile[0])+'.png',bbox_inches='tight',dpi=100) # dpi=1000
    plt.close(fig)
    

    # 9. calculations
    #quantitative assessment of unmapped footprint area. Assumes a completeness ratio > 0.5 is mapped
    builtUpIdx = np.nonzero(np.asarray(vals)>=noRoadThresh)
    builtUp = vals2[builtUpIdx]

    areasBuilt = np.asarray(vals)[builtUpIdx]

    completeIdx = np.nonzero(builtUp>=0.5)
    incompleteIdx = np.nonzero(builtUp<0.5)
    completeAreas = areasBuilt[completeIdx]
    incompleteAreas = areasBuilt[incompleteIdx]
    complete = builtUp[completeIdx]
    incomplete = builtUp[incompleteIdx]
    mappedBuildings=np.sum(completeAreas)
    unmappedBuildings=np.sum(incompleteAreas)
    
    mapped_list.append(mappedBuildings)
    mapped_need_list.append(unmappedBuildings)

In [ ]:
# create dataframe to show mapped / unmapped / %mapped for each chunk

In [ ]:
df = pd.DataFrame((list(zip(mapped_list, mapped_need_list))))
df.columns = ['mapped', 'unmapped']
df['%mapped'] = 100*df['mapped']/(df['mapped']+df['unmapped'])

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler() # object
#values between 0 and 1
norm = scaling.fit_transform(df[['%mapped']]) # pass only certain columns 
df['%mapped_norm'] = norm

In [ ]:
df.describe()

In [ ]:
# save df
df.to_csv('../data/4_applymodel/grid_03/final_calculations.csv')

In [ ]:
# check distribution
hist = df.hist(figsize=(15,10), bins=10)
hist